In [27]:
# calculate the difference between the files in test_data and test_data3 that end in 2024-12-12 - 2024-12-19
import pandas as pd
import os
dir_ = "test_data/sp500/"
dir_3 = "test_data3/sp500/"
count = 0

file1_list = []
file2_list = []

df1_list = []
df2_list = []	

for file in os.listdir(dir_):
    if file.split("_")[1] in ["2024-12-12", "2024-12-13"]:
        file1_list.append(file)

for file in os.listdir(dir_3):
    if file.split("_")[1] in ["2024-12-18", "2024-12-19"]:
        file2_list.append(file)

file1_list.sort(key=lambda x: x.split("_")[2].split(".csv")[0])
file2_list.sort(key=lambda x: x.split("_")[2].split(".csv")[0])

for file in file1_list:
    file_df = pd.read_csv(os.path.join(dir_, file), index_col=0)
    file_df = file_df[(file_df.index >= "2024-01-01") & (file_df.index <= "2024-12-12")]
    df1_list.append(file_df)

for file in file2_list:
    file_df = pd.read_csv(os.path.join(dir_3, file), index_col=0)
    file_df = file_df[(file_df.index >= "2024-01-01") & (file_df.index <= "2024-12-12")]
    df2_list.append(file_df)



In [ ]:
for df1, df2 in zip(df1_list, df2_list):
    print(df1["Adj Close"].equals(df2["Adj Close"]))
    if not df1["Adj Close"].equals(df2["Adj Close"]):
        # get the difference
        for i in range(len(df1)):
            if df1["Adj Close"].iloc[i] != df2["Adj Close"].iloc[i]:
                print(df1["Adj Close"].iloc[i], df2["Adj Close"].iloc[i])

    break

In [25]:
import pandas as pd
import utils

df = utils.get_repo_data("sp500_companies.csv")
# get last row value
last_value = df.iloc[-1]["tickers"]
last_value.count(",")

502

In [ ]:
import utils
utils.load_sp500_data("test_data2/")
#TODO load date count.csv

In [3]:
# check for empty files and add them to a replacement list
import os
import pandas as pd

print(len(os.listdir("test_data/sp500")))
# iterate over all csv files in test_data/sp500 folder
for file in os.listdir("test_data/sp500"):
    if file.endswith(".csv"):
        # read csv file
        df = pd.read_csv(f"test_data/sp500/{file}", header=[0,1], index_col=0)
        # get ticker symbol from the header
        ticker = df.columns.levels[1][0]
        if df.empty:
            # add ticker to replacement list
            with open("replacement_list.csv", "a") as f:
                f.write(f"{ticker},\n")
            # remove file
            os.remove(f"test_data/sp500/{file}")

728


In [32]:
# calculate stock price breadth
import pandas as pd
import os

# some date way in the past to load all data
last_date = "2024-01-01"
sp500_dir = "test_data/sp500/"

# get last date of the data
old_df = pd.read_csv("breadth_ratio_tmp.csv", index_col=0)
last_date = old_df.index[-1]
date_count_data = pd.read_csv("date_count.csv", index_col=0)
#new_date = (pd.to_datetime(last_date).date() + pd.DateOffset(days=1)).strftime("%Y-%m-%d")

df_list = []
# iterate over all csv files in test_data/sp500 folder
for file in os.listdir(sp500_dir):
    if file.endswith(".csv"):
        # read csv file
        df = pd.read_csv(f"{sp500_dir}{file}", header=[0,1], index_col=0)
        if df.index[-1] < last_date:
            continue
        file_start_date = file.split("_")[0]
        file_start_date = pd.to_datetime(file_start_date).date() + pd.DateOffset(days=365)
        first_relevant_date = max(file_start_date.strftime("%Y-%m-%d"), last_date)
        # init date is the date that needs to be added to calculate the difference for the first date (the first date before the first relevant date in date_count_data)
        init_date = date_count_data.index[(date_count_data.index < first_relevant_date)][-1]
        # use date_count_data to add missing dates between the first relevant date and the last date
        df = df.reindex(date_count_data.index[(date_count_data.index <= df.index[-1])], fill_value=None)
        # fill any nans after the first valid value but before the last valid value
        # Identify the indices to forward-fill
        ffill_mask = df.notna() | df.bfill().notna()
        # Forward-fill the data
        df = df.ffill().where(ffill_mask)
        # remove all irrelevant data
        df = df[df.index >= init_date]
        df_list.append(df)

# Process each ticker
for df in df_list:
    # Calculate the daily price change
    df['Change'] = df['Adj Close'].diff()
    
    # Define volumes for "up" and "down" days
    df['Volume Up'] = df['Volume'].where(df['Change'] > 0, 0)  # Volume for "up" days
    df['Volume Down'] = df['Volume'].where(df['Change'] < 0, 0)  # Volume for "down" days

if not df_list:
    print("No new data to process")
else:
    # Combine all tickers' data
    combined_breadth = pd.concat(df_list).groupby(level=0).sum()
    # Calculate total breadth
    res = combined_breadth['Volume Up'] / (combined_breadth['Volume Down'] + combined_breadth['Volume Up'])
    res.fillna(0.5, inplace=True)
    res = pd.DataFrame(res, columns=['Breadth Ratio'], index=combined_breadth.index)
    res.to_csv("breadth_ratio_tmp4.csv")

In [2]:
# plot breadth ratio tmp2.csv and breadth_ratio csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TkAgg')
df = pd.read_csv("breadth_ratio_tmp2.csv", index_col=0)
df2 = pd.read_csv("repoData/breadth_ratio.csv", index_col=0)
df.index = pd.to_datetime(df.index)
df2.index = pd.to_datetime(df2.index)
plt.plot(df.index, df['Breadth Ratio'], label="tmp2")
plt.plot(df2.index, df2['Breadth Ratio'], label="repo")
plt.legend()
plt.show()



In [17]:
# plot breadth_ratio and breadth_ratio2 in one graph
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import utils
matplotlib.use('tkagg')

df2 = pd.read_csv("breadth_ratio2.csv", index_col=0)
df2.index = pd.to_datetime(df2.index)
# moving average
#plt.plot(utils.normalize_tanh(df2.ewm(span=10).mean(), steepness=7, shift=-0.5), color='red')
plt.plot(utils.normalize_tanh(df2.ewm(span=100, min_periods=50).mean() * 100, steepness=0.3, shift=-50), color='red')
plt.show()

In [ ]:
import os
import pandas as pd

# df_list = []
# # iterate over all csv files in test_data/sp500 folder
# for file in os.listdir("test_data/sp500"):
#     if file.endswith(".csv"):
#         # read csv file
#         df = pd.read_csv(f"test_data/sp500/{file}", header=[0,1], index_col=0)
#         file_start_date = file.split("_")[0]
#         file_start_date = pd.to_datetime(file_start_date).date() + pd.DateOffset(days=365)
#         # remove all rows before 1 year from the start date
#         df = df[df.index >= file_start_date.strftime("%Y-%m-%d")]
#         df_list.append(df)

# # concat all dataframes while keeping nan values
# combined_breadth = pd.concat(df_list, axis=1, join='outer')
combined_breadth = pd.read_csv("combined_breadth.csv", index_col=0, header=[0,1])
data_adj_close = combined_breadth.xs('Adj Close', axis=1, level=0)
# fill any nans after the first valid value but before the last valid value
# Identify the indices to forward-fill
ffill_mask = data_adj_close.notna() | data_adj_close.bfill().notna()
# Forward-fill the data
data_adj_close = data_adj_close.ffill().where(ffill_mask)
# period is the number of days to look back for a 52 week high
# 1. Calculate the 52 week highs and lows
year_highs = data_adj_close.rolling(window=252, min_periods=1).max()
year_lows = data_adj_close.rolling(window=252, min_periods=1).min()
# 2. Calculate the high and low of the last period days
period_highs = data_adj_close.rolling(window=10, min_periods=1).max()
period_lows = data_adj_close.rolling(window=10, min_periods=1).min()
# 3. Calculate the number of stocks that are at their 52 week high and low
num_highs = ((data_adj_close == period_highs) & (data_adj_close == year_highs)).sum(axis=1)
num_lows = ((data_adj_close == period_lows) & (data_adj_close == year_lows)).sum(axis=1)
# 4. Calculate the stock price strength
stock_price_strength = num_highs / (num_highs + num_lows) * 100
# replace NaN values with 50 (no stocks at 52 week high or low)
stock_price_strength.fillna(50, inplace=True)
df = pd.DataFrame((num_highs, num_lows, stock_price_strength), index=["Num Highs", "Num Lows", "Stock Price Strength"]).T
df

## Stock Price Strength

In [4]:
from sys import exit
import pandas as pd
import os
import yfinance as yf
# the number of days to look back for a high or low to count
period = 14
num_highs = {}
num_lows = {}
if not os.path.exists("test_data/trading_days.csv"):
    yf.download("^GSPC", start="1995-01-01").to_csv("test_data/trading_days.csv")
trading_days = pd.read_csv("test_data/trading_days.csv", index_col=0, header=[0,1])
trading_days.index = pd.to_datetime(trading_days.index)
for date in trading_days.index:
    num_highs[date] = 0
    num_lows[date] = 0
# iterate over all csv files in test_data/sp500 folder
for file in os.listdir("test_data/sp500"):
    if file.endswith(".csv"):
        # read csv file
        df = pd.read_csv(f"test_data/sp500/{file}", header=[0,1], index_col=0, parse_dates=True)["Adj Close"]
        df.index = pd.to_datetime(df.index)
        file_start_date = file.split("_")[0]
        actual_start_date = pd.to_datetime(file_start_date).date() + pd.DateOffset(days=365)
        # add all trading days that are missing in the dataframe
        df = df.reindex(trading_days.index, axis=0)
        # Identify the indices to forward-fill
        ffill_mask = df.notna() | df.bfill().notna()
        # Forward-fill the data
        df = df.ffill().where(ffill_mask)
        df.dropna(inplace=True)
        # get row that is after actual_start_date
        first_valid_row = df[df.index >= actual_start_date].iloc[0]
        year_high = df.loc[df[df.index <= first_valid_row.name].idxmax()].iloc[0]
        year_low = df.loc[df[df.index <= first_valid_row.name].idxmin()].iloc[0]
        current_row_index = df.index.get_loc(first_valid_row.name)
        current_row = first_valid_row
        while current_row_index < len(df) - 1 and current_row.notna().all():
            if current_row.iloc[0] >= year_high.iloc[0]:
                year_high = current_row
            elif current_row.iloc[0] <= year_low.iloc[0]:
                year_low = current_row
            else:
                one_year_ago = current_row.name - pd.DateOffset(days=365)
                if year_high.name < one_year_ago:
                    # calculate year_high again where the date is less than one year ago from the current date
                    year_high = df.loc[df[(df.index >= one_year_ago) & (df.index <= current_row.name)].idxmax()].iloc[0]
                if year_low.name < one_year_ago:
                    # calculate year_low again
                    year_low = df.loc[df[(df.index >= one_year_ago) & (df.index <= current_row.name)].idxmin()].iloc[0]
            if current_row.name - pd.DateOffset(days=period) <= year_high.name:
                num_highs[current_row.name] += 1
            if current_row.name - pd.DateOffset(days=period) <= year_low.name:
                num_lows[current_row.name] += 1
            current_row_index += 1
            current_row = df.iloc[current_row_index]
        # TODO try percentage to low or high (e.g. (high - low)*0.2 distance (20 low, 70 high -> 20-30 low 60-70 high), 20 low 25 high > 20-21 low 24-25 high)
        #compare with price_strength1.csv and price_strength2.csv

In [7]:
from sys import exit
import pandas as pd
import os
import yfinance as yf
num_highs = {}
num_lows = {}
if not os.path.exists("test_data/trading_days.csv"):
    yf.download("^GSPC", start="1995-01-01").to_csv("test_data/trading_days.csv")
trading_days = pd.read_csv("test_data/trading_days.csv", index_col=0, header=[0,1])
trading_days.index = pd.to_datetime(trading_days.index)
for date in trading_days.index:
    num_highs[date] = 0
    num_lows[date] = 0
# iterate over all csv files in test_data/sp500 folder
for file in os.listdir("test_data/sp500"):
    if file.endswith(".csv"):
        # read csv file
        df = pd.read_csv(f"test_data/sp500/{file}", header=[0,1], index_col=0, parse_dates=True)["Adj Close"]
        df.index = pd.to_datetime(df.index)
        file_start_date = file.split("_")[0]
        actual_start_date = pd.to_datetime(file_start_date).date() + pd.DateOffset(days=365)
        # add all trading days that are missing in the dataframe
        df = df.reindex(trading_days.index, axis=0)
        # Identify the indices to forward-fill
        ffill_mask = df.notna() | df.bfill().notna()
        # Forward-fill the data
        df = df.ffill().where(ffill_mask)
        df.dropna(inplace=True)
        # get row that is after actual_start_date
        first_valid_row = df[df.index >= actual_start_date].iloc[0]
        year_high = df.loc[df[df.index <= first_valid_row.name].idxmax()].iloc[0]
        year_low = df.loc[df[df.index <= first_valid_row.name].idxmin()].iloc[0]
        current_row_index = df.index.get_loc(first_valid_row.name)
        current_row = first_valid_row
        while current_row_index < len(df) - 1 and current_row.notna().all():
            if current_row.iloc[0] >= year_high.iloc[0]:
                year_high = current_row
            elif current_row.iloc[0] <= year_low.iloc[0]:
                year_low = current_row
            else:
                one_year_ago = current_row.name - pd.DateOffset(days=365)
                if year_high.name < one_year_ago:
                    # calculate year_high again where the date is less than one year ago from the current date
                    year_high = df.loc[df[(df.index >= one_year_ago) & (df.index <= current_row.name)].idxmax()].iloc[0]
                if year_low.name < one_year_ago:
                    # calculate year_low again
                    year_low = df.loc[df[(df.index >= one_year_ago) & (df.index <= current_row.name)].idxmin()].iloc[0]
            if current_row.iloc[0] >= year_high.iloc[0] - (year_high.iloc[0] - year_low.iloc[0]) * 0.2:
                num_highs[current_row.name] += 1
            if current_row.iloc[0] <= year_low.iloc[0] + (year_high.iloc[0] - year_low.iloc[0]) * 0.2:
                num_lows[current_row.name] += 1
            current_row_index += 1
            current_row = df.iloc[current_row_index]
        # TODO try percentage to low or high (e.g. (high - low)*0.2 distance (20 low, 70 high -> 20-30 low 60-70 high), 20 low 25 high > 20-21 low 24-25 high)
        #compare with price_strength1.csv and price_strength2.csv

In [8]:
df = pd.DataFrame((num_highs, num_lows), index=["Num Highs", "Num Lows"]).T
df["Strength"] = df["Num Highs"] / (df["Num Highs"] + df["Num Lows"]) * 100
df.to_csv("price_strength4.csv")

## Stock Price Breadth

1. do not count unchanged price volumes
2. test if pd.concat works correctly (missing values, ranges, zeros, etc.)